# Exploration of simple pull request (PR) agent

## Intro
How to go from LLM response/answer to a commit (and PR)?

- One way could be to create an agent. It could extract only relevant text in
an answer and then use this to make the commit for a config task.
- Another way could be to try playing around with the prompt to get a more
exact/concise output.
- Exploring possibility simple (A)ST-based indexing of source files.
Ensures we keep track of the actual lines in source file of a snippet. Feed
al this context forward into the LLM. Maybe it can use the additional (e.g.
line number) context to be more precise in it's output.

## Goal
Create an agent/chain to extract relevant code/text from LLM answer and
create a commit.